In [1]:
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool
import os
import logging
from sklearn.model_selection import train_test_split

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
path_data_folder = Path.cwd().parent.parent / 'data'
print(path_data_folder)

folder_raw_data_milling = path_data_folder / "raw/milling"
folder_interim_data_milling = path_data_folder / "interim/milling"
folder_processed_data_milling = path_data_folder / "processed/milling"

/home/tim/Documents/feat-store/data


In [3]:
df_raw = pd.read_csv(folder_raw_data_milling / "milling.csv.gz", compression='gzip',)
df_raw.head()

,cut_id,cut_no,case,time,ae_spindle,ae_table,vib_spindle,vib_table,smcdc,smcac,tool_class
0,0_0,0,1,0.000,0.219727,0.272827,0.733643,2.116699,6.840820,0.124512,0
1,0_0,0,1,0.004,0.246582,0.322266,0.778809,2.277832,6.660156,-0.561523,0
2,0_0,0,1,0.008,0.294189,0.283813,0.758057,2.343750,6.508789,-2.099609,0
3,0_0,0,1,0.012,0.323486,0.260010,0.726318,2.448730,6.542969,-2.731934,0
4,0_0,0,1,0.016,0.290527,0.253296,0.653076,2.546387,6.621094,-3.505859,0


In [5]:
# add column for the y label, where 1 is the label for the positive class when the tool_class > 1
df_raw['y'] = df_raw['tool_class'] > 1
df_raw['y'] = df_raw['y'].astype(int)
df_raw.head()

,cut_id,cut_no,case,time,ae_spindle,ae_table,vib_spindle,vib_table,smcdc,smcac,tool_class,y
0,0_0,0,1,0.000,0.219727,0.272827,0.733643,2.116699,6.840820,0.124512,0,0
1,0_0,0,1,0.004,0.246582,0.322266,0.778809,2.277832,6.660156,-0.561523,0,0
2,0_0,0,1,0.008,0.294189,0.283813,0.758057,2.343750,6.508789,-2.099609,0,0
3,0_0,0,1,0.012,0.323486,0.260010,0.726318,2.448730,6.542969,-2.731934,0,0
4,0_0,0,1,0.016,0.290527,0.253296,0.653076,2.546387,6.621094,-3.505859,0,0


In [29]:
df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip',)
print(df.shape)
df.head()

(11570, 4510)


,cut_id,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,...,smcac__query_similarity_count__query_None__threshold_0.0,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class
0,50_23,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,...,NaN,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248,50,4,1
1,50_24,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,...,NaN,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660,50,4,1
2,50_25,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,...,NaN,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001,50,4,1
3,50_26,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,...,NaN,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085,50,4,1
4,50_27,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,...,NaN,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327,50,4,1


In [30]:
df = df[df['case']==15]

df['y'] = df['tool_class'] > 1
df['y'] = df['y'].astype(int)
print(df.shape)
df.head()

(520, 4511)


,cut_id,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,...,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class,y
10654,154_0,3.277600e-05,3.138502e-07,-0.000004,0.002755,0.002738,0.002713,9.108564,0.103335,0.0,...,1.210316,9.495060,3.155575,1.471686,1.409514,4.198592,154,15,0,0
10655,154_1,-3.481216e-05,-4.725636e-05,-0.000036,0.002773,0.002761,0.002748,9.774098,0.106730,0.0,...,1.145610,6.663433,1.933154,1.306129,1.204335,1.378873,154,15,0,0
10656,154_10,-4.339720e-07,2.743999e-06,0.000019,0.002787,0.002761,0.002742,9.451535,0.103611,0.0,...,1.267046,11.902700,5.012953,4.047941,1.298777,8.912656,154,15,0,0
10657,154_11,1.703928e-05,3.375236e-05,0.000009,0.002836,0.002819,0.002831,10.165892,0.119084,0.0,...,1.365485,12.961481,6.541192,6.282037,1.916424,10.629192,154,15,0,0
10658,154_12,-8.043911e-06,-3.024958e-07,-0.000004,0.002694,0.002670,0.002650,8.906037,0.113346,0.0,...,1.257279,10.932990,4.016544,1.559468,1.285410,6.817448,154,15,0,0


In [31]:
# drop any columns with NaN values
df = df.dropna(axis=1, how='any')
df.set_index('cut_id', inplace=True)

print(df.shape)
df.head()

(520, 2803)


,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,ae_spindle__symmetry_looking__r_0.05,...,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class,y
cut_id,,,,,,,,,,,,,,,,,,,,,
154_0,3.277600e-05,3.138502e-07,-0.000004,0.002755,0.002738,0.002713,9.108564,0.103335,0.0,1.0,...,1.210316,9.495060,3.155575,1.471686,1.409514,4.198592,154,15,0,0
154_1,-3.481216e-05,-4.725636e-05,-0.000036,0.002773,0.002761,0.002748,9.774098,0.106730,0.0,0.0,...,1.145610,6.663433,1.933154,1.306129,1.204335,1.378873,154,15,0,0
154_10,-4.339720e-07,2.743999e-06,0.000019,0.002787,0.002761,0.002742,9.451535,0.103611,0.0,1.0,...,1.267046,11.902700,5.012953,4.047941,1.298777,8.912656,154,15,0,0
154_11,1.703928e-05,3.375236e-05,0.000009,0.002836,0.002819,0.002831,10.165892,0.119084,0.0,1.0,...,1.365485,12.961481,6.541192,6.282037,1.916424,10.629192,154,15,0,0
154_12,-8.043911e-06,-3.024958e-07,-0.000004,0.002694,0.002670,0.002650,8.906037,0.113346,0.0,1.0,...,1.257279,10.932990,4.016544,1.559468,1.285410,6.817448,154,15,0,0


In [35]:
df = df.drop(['case', 'tool_class', 'y', 'cut_no'], axis=1)
df.head()

,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,ae_spindle__symmetry_looking__r_0.05,...,smcac__permutation_entropy__dimension_4__tau_1,smcac__permutation_entropy__dimension_5__tau_1,smcac__permutation_entropy__dimension_6__tau_1,smcac__permutation_entropy__dimension_7__tau_1,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98"
cut_id,,,,,,,,,,,,,,,,,,,,,
154_0,3.277600e-05,3.138502e-07,-0.000004,0.002755,0.002738,0.002713,9.108564,0.103335,0.0,1.0,...,1.947155,2.458517,2.871755,3.184946,1.210316,9.495060,3.155575,1.471686,1.409514,4.198592
154_1,-3.481216e-05,-4.725636e-05,-0.000036,0.002773,0.002761,0.002748,9.774098,0.106730,0.0,0.0,...,2.074538,2.553512,2.975967,3.298287,1.145610,6.663433,1.933154,1.306129,1.204335,1.378873
154_10,-4.339720e-07,2.743999e-06,0.000019,0.002787,0.002761,0.002742,9.451535,0.103611,0.0,1.0,...,1.908081,2.360681,2.757294,3.136191,1.267046,11.902700,5.012953,4.047941,1.298777,8.912656
154_11,1.703928e-05,3.375236e-05,0.000009,0.002836,0.002819,0.002831,10.165892,0.119084,0.0,1.0,...,2.064300,2.490675,2.817898,3.131181,1.365485,12.961481,6.541192,6.282037,1.916424,10.629192
154_12,-8.043911e-06,-3.024958e-07,-0.000004,0.002694,0.002670,0.002650,8.906037,0.113346,0.0,1.0,...,2.091574,2.565142,2.939898,3.276163,1.257279,10.932990,4.016544,1.559468,1.285410,6.817448


In [32]:
dfy = df['y']
dfy.head()

cut_id
154_0     0
154_1     0
154_10    0
154_11    0
154_12    0
Name: y, dtype: int64

# Test Selection

In [71]:
from src.features.select_feat_from_dataframe import milling_select_features, milling_add_y_label_anomaly, milling_create_data_splits

In [70]:
# load first 500 rows of data from milling_features.csv.gz
# df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip', nrows=500)
df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip',)
print(df.shape)
df.head()

(11570, 4510)


,cut_id,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,...,smcac__query_similarity_count__query_None__threshold_0.0,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class
0,50_23,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,...,NaN,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248,50,4,1
1,50_24,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,...,NaN,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660,50,4,1
2,50_25,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,...,NaN,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001,50,4,1
3,50_26,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,...,NaN,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085,50,4,1
4,50_27,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,...,NaN,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327,50,4,1


In [80]:
df_train, df_val, df_test = milling_create_data_splits(df)

df_feat_train shape: (6942, 4510)
df_feat_test shape: (2314, 4510)
df_feat_val shape: (2314, 4510)


In [77]:
# count the number values for each 'tool_class' in df_train
df_train['tool_class'].value_counts()

1    3900
0    2536
2     506
Name: tool_class, dtype: int64

In [79]:
df_val['tool_class'].value_counts()

1    1300
0     845
2     169
Name: tool_class, dtype: int64

In [81]:
df_val[df_val['tool_class'] == 2]['case'].unique()

array([13, 11, 15, 10, 14,  5,  9])

In [69]:


df = milling_add_y_label_anomaly(df)
df = milling_select_features(df)


In [66]:
df.head()

,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,ae_spindle__symmetry_looking__r_0.05,...,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class,y
cut_id,,,,,,,,,,,,,,,,,,,,,
50_23,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,1.0,...,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248,50,4,1,0
50_24,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,1.0,...,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660,50,4,1,0
50_25,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,1.0,...,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001,50,4,1,0
50_26,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,1.0,...,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085,50,4,1,0
50_27,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,1.0,...,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327,50,4,1,0


# Split on Case Number

In [4]:
df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip',)
print(df.shape)
df.head()

(11570, 4510)


,cut_id,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,...,smcac__query_similarity_count__query_None__threshold_0.0,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class
0,50_23,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,...,NaN,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248,50,4,1
1,50_24,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,...,NaN,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660,50,4,1
2,50_25,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,...,NaN,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001,50,4,1
3,50_26,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,...,NaN,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085,50,4,1
4,50_27,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,...,NaN,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327,50,4,1


In [5]:
for i in df['case'].unique():
    print(i)


4
9
10
11
12
3
13
14
1
2
5
6
7
8
15
16


In [6]:
int(df[df['cut_no']==7]['tool_class'].unique())

1

In [7]:
# dictionary comprehension to create dict of each cut_no and its corresponding tool_class
cut_id_tool_class_dict = {i : int(df[df['cut_no']==i]['tool_class'].unique()) for i in df['cut_no'].unique()}

In [8]:
cut_numbers = []
tool_classes = []
for i in cut_id_tool_class_dict:
    cut_numbers.append(i)
    tool_classes.append(cut_id_tool_class_dict[i])


cut_numbers_train, cut_numbers_test, tool_classes_train, tool_classes_test = train_test_split(
    np.array(cut_numbers), np.array(tool_classes),  test_size=0.6, random_state=13, stratify=np.array(tool_classes)
)

In [14]:
print(len(cut_numbers_train))
cut_numbers_train

66


array([ 49, 107,  93, 154,  36,  60,  34,  52,  44,  55,  70,  82,  67,
        84, 120, 100, 144, 147,  15,   1,   3,  69,  64, 140,   2, 139,
        85,   7, 150,  63,  47,  61,  18, 119,  62,  81,  12,  78,  14,
        43, 157,  19,  13,   9, 110,  24, 145, 101,  87,  66,  91, 111,
       105,  77, 161,  35,  51,  21, 165, 138, 118,   4, 123,  26,  80,
        37])

In [18]:
cut_id_tool_class_dict[80]

0

In [15]:
print(len(tool_classes_train))
tool_classes_train

66


array([1, 1, 2, 0, 1, 2, 0, 0, 1, 0, 2, 1, 1, 1, 1, 1, 2, 0, 1, 0, 0, 1,
       0, 2, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1])

In [23]:
# get the number of values for each number in tool_classes_train array
print(np.unique(tool_classes_train, return_counts=True))


(array([0, 1, 2]), array([24, 37,  5]))


In [36]:
from tsfresh import select_features

features_filtered = select_features(df, dfy)

In [37]:
features_filtered.shape

(520, 1079)

In [38]:
features_filtered.head()

,smcac__fourier_entropy__bins_5,smcdc__large_standard_deviation__r_0.1,smcdc__large_standard_deviation__r_0.05,smcdc__symmetry_looking__r_0.9500000000000001,smcdc__symmetry_looking__r_0.9,smcdc__symmetry_looking__r_0.8500000000000001,smcdc__symmetry_looking__r_0.2,smcdc__symmetry_looking__r_0.8,smcdc__large_standard_deviation__r_0.15000000000000002,smcdc__symmetry_looking__r_0.25,...,ae_spindle__permutation_entropy__dimension_5__tau_1,ae_table__autocorrelation__lag_2,"vib_spindle__cwt_coefficients__coeff_6__w_10__widths_(2, 5, 10, 20)","ae_spindle__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0",smcac__friedrich_coefficients__coeff_2__m_3__r_30,"smcac__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.2","vib_spindle__cwt_coefficients__coeff_9__w_10__widths_(2, 5, 10, 20)","ae_table__cwt_coefficients__coeff_1__w_10__widths_(2, 5, 10, 20)","vib_spindle__cwt_coefficients__coeff_3__w_5__widths_(2, 5, 10, 20)","smcac__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.6"
cut_id,,,,,,,,,,,,,,,,,,,,,
154_0,0.270641,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.767818,-0.062210,0.370336,0.000020,0.075035,0.075309,0.433727,0.047193,0.276993,0.003746
154_1,0.270641,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.767818,0.021735,0.432049,0.000002,0.038799,0.097216,0.511970,0.028897,0.273565,0.024083
154_10,0.270641,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.932610,0.051142,0.426522,0.000037,0.058041,0.064027,0.497278,0.064209,0.287292,0.018695
154_11,0.270641,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.817086,-0.047990,0.369990,0.000063,0.112647,0.123362,0.460669,0.047726,0.243635,0.032612
154_12,0.270641,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.767818,-0.078948,0.401199,0.000026,0.004600,0.077711,0.456518,0.039298,0.300572,0.036906


In [39]:
df['smcdc__large_standard_deviation__r_0.15000000000000002']

cut_id
154_0     1.0
154_1     1.0
154_10    1.0
154_11    1.0
154_12    1.0
         ... 
160_72    0.0
160_73    0.0
160_74    0.0
160_8     0.0
160_9     0.0
Name: smcdc__large_standard_deviation__r_0.15000000000000002, Length: 520, dtype: float64

In [40]:
from tsfresh.feature_extraction import settings
kind_to_fc_parameters = settings.from_columns(features_filtered)

In [41]:
kind_to_fc_parameters

{'smcac': {'fourier_entropy': [{'bins': 5},
   {'bins': 3},
   {'bins': 10},
   {'bins': 100},
   {'bins': 2}],
  'fft_coefficient': [{'attr': 'abs', 'coeff': 4},
   {'attr': 'abs', 'coeff': 5},
   {'attr': 'abs', 'coeff': 6},
   {'attr': 'abs', 'coeff': 2},
   {'attr': 'abs', 'coeff': 8},
   {'attr': 'abs', 'coeff': 15},
   {'attr': 'abs', 'coeff': 9},
   {'attr': 'abs', 'coeff': 7},
   {'attr': 'abs', 'coeff': 12},
   {'attr': 'abs', 'coeff': 13},
   {'attr': 'abs', 'coeff': 17},
   {'attr': 'abs', 'coeff': 26},
   {'attr': 'abs', 'coeff': 16},
   {'attr': 'abs', 'coeff': 21},
   {'attr': 'abs', 'coeff': 29},
   {'attr': 'abs', 'coeff': 30},
   {'attr': 'abs', 'coeff': 14},
   {'attr': 'abs', 'coeff': 10},
   {'attr': 'abs', 'coeff': 20},
   {'attr': 'abs', 'coeff': 31},
   {'attr': 'abs', 'coeff': 27},
   {'attr': 'abs', 'coeff': 11},
   {'attr': 'abs', 'coeff': 19},
   {'attr': 'abs', 'coeff': 22},
   {'attr': 'abs', 'coeff': 1},
   {'attr': 'abs', 'coeff': 28},
   {'attr': 'abs', 

In [47]:
import json
with open('kind_to_fc_parameters.json', 'w') as fp:
    json.dump(kind_to_fc_parameters, fp)


In [48]:
with open('kind_to_fc_parameters.json', 'r') as fp:
    data = json.load(fp)

In [52]:
path_data_folder = Path.cwd().parent.parent / 'data'
path_data_folder

PosixPath('/home/tim/Documents/feat-store/data')

In [53]:
folder_raw_data_milling = path_data_folder / "raw/milling"
folder_interim_data_milling = path_data_folder / "interim/milling"
folder_processed_data_milling = path_data_folder / "processed/milling"

In [54]:
fc_parameters = {
    "length": None,
    "kurtosis": None,
}

In [55]:
df = pd.read_csv(folder_raw_data_milling / "milling.csv.gz", compression='gzip',)
df.head()

,cut_id,case,time,ae_spindle,ae_table,vib_spindle,vib_table,smcdc,smcac,tool_class
0,0_0,0,0.000,0.219727,0.272827,0.733643,2.116699,6.840820,0.124512,0
1,0_0,0,0.004,0.246582,0.322266,0.778809,2.277832,6.660156,-0.561523,0
2,0_0,0,0.008,0.294189,0.283813,0.758057,2.343750,6.508789,-2.099609,0
3,0_0,0,0.012,0.323486,0.260010,0.726318,2.448730,6.542969,-2.731934,0
4,0_0,0,0.016,0.290527,0.253296,0.653076,2.546387,6.621094,-3.505859,0


In [56]:
df.columns

Index(['cut_id', 'case', 'time', 'ae_spindle', 'ae_table', 'vib_spindle',
       'vib_table', 'smcdc', 'smcac', 'tool_class'],
      dtype='object')

In [57]:
df_raw_labels = df[['cut_id', 'case', 'tool_class']].drop_duplicates().copy()
print(df_raw_labels.shape)
df_raw_labels.head()

(11570, 3)


,cut_id,case,tool_class
0,0_0,0,0
64,0_1,0,0
128,0_2,0,0
192,0_3,0,0
256,0_4,0,0


In [46]:
df = df.drop(columns=['case', 'tool_class'])[:500]
df.head()

,cut_id,time,ae_spindle,ae_table,vib_spindle,vib_table,smcdc,smcac
0,0_0,0.000,0.219727,0.272827,0.733643,2.116699,6.840820,0.124512
1,0_0,0.004,0.246582,0.322266,0.778809,2.277832,6.660156,-0.561523
2,0_0,0.008,0.294189,0.283813,0.758057,2.343750,6.508789,-2.099609
3,0_0,0.012,0.323486,0.260010,0.726318,2.448730,6.542969,-2.731934
4,0_0,0.016,0.290527,0.253296,0.653076,2.546387,6.621094,-3.505859


In [47]:
df_feat = extract_features(
    df,
    column_id="cut_id",
    column_sort="time",
    default_fc_parameters=fc_parameters,
    disable_progressbar=False,
)

df_feat = df_feat.reset_index().rename(columns={'index':'cut_id'})
df_feat.merge(df_raw_labels, on='cut_id', how='left')

Feature Extraction: 100%|██████████| 16/16 [00:00<00:00, 1631.59it/s]


,cut_id,ae_spindle__length,ae_spindle__kurtosis,ae_table__length,ae_table__kurtosis,vib_spindle__length,vib_spindle__kurtosis,vib_table__length,vib_table__kurtosis,smcdc__length,smcdc__kurtosis,smcac__length,smcac__kurtosis,case,tool_class
0,0_0,64.0,-0.348446,64.0,-0.190215,64.0,-0.126833,64.0,2.144662,64.0,-0.256599,64.0,-1.367461,0,0
1,0_1,64.0,0.724161,64.0,-0.236114,64.0,0.504015,64.0,2.357173,64.0,0.384966,64.0,-1.248971,0,0
2,0_2,64.0,-0.297622,64.0,-0.425231,64.0,0.486500,64.0,2.844314,64.0,0.052151,64.0,-1.297844,0,0
3,0_3,64.0,0.651310,64.0,-0.494050,64.0,-0.428992,64.0,2.177012,64.0,0.004801,64.0,-1.197047,0,0
4,0_4,64.0,0.561234,64.0,1.058162,64.0,0.526977,64.0,3.088219,64.0,-0.053338,64.0,-1.395728,0,0
5,0_5,64.0,0.856951,64.0,0.157097,64.0,-0.082165,64.0,1.998757,64.0,-0.104125,64.0,-1.206616,0,0
6,0_6,64.0,-0.228500,64.0,-0.080787,64.0,-0.128798,64.0,2.419692,64.0,-0.159358,64.0,-1.472339,0,0
7,0_7,52.0,0.296168,52.0,-0.066365,52.0,0.527285,52.0,2.525260,52.0,-0.706810,52.0,-0.879845,0,0


In [70]:
fc_parameters = {
    "length": None,
    # "kurtosis": None,
}

folder_raw_data_milling = path_data_folder / "raw/milling"
folder_interim_data_milling = path_data_folder / "interim/milling"
folder_processed_data_milling = path_data_folder / "processed/milling"

# if interim & processed folders don't exist, create them
Path(folder_interim_data_milling).mkdir(parents=True, exist_ok=True)
Path(folder_processed_data_milling).mkdir(parents=True, exist_ok=True)

# read in raw milling data to a pandas dataframe
df = pd.read_csv(folder_raw_data_milling / "milling.csv.gz", compression='gzip',)
print("df.columns", df.columns)

print("Shape of df:", df.shape)

# for testing purposes only include some cuts
# df = df[df["cut_id"].isin(['0_0', '0_1', '0_2'])]

df_feat = milling_features(df, 1, 0, feature_dictionary=fc_parameters)

df.columns Index(['cut_id', 'case', 'time', 'ae_spindle', 'ae_table', 'vib_spindle',
       'vib_table', 'smcdc', 'smcac', 'tool_class'],
      dtype='object')
Shape of df: (740480, 10)


Feature Extraction: 100%|██████████| 20/20 [00:09<00:00,  2.10it/s]


In [71]:
df_feat.head()

,cut_id,ae_spindle__length,ae_table__length,vib_spindle__length,vib_table__length,smcdc__length,smcac__length,case,tool_class
0,0_0,64.0,64.0,64.0,64.0,64.0,64.0,0,0
1,0_1,64.0,64.0,64.0,64.0,64.0,64.0,0,0
2,0_10,64.0,64.0,64.0,64.0,64.0,64.0,0,0
3,0_11,64.0,64.0,64.0,64.0,64.0,64.0,0,0
4,0_12,64.0,64.0,64.0,64.0,64.0,64.0,0,0


In [4]:
df = pd.read_csv(folder_interim_data_milling / "milling_0.csv")
print(df.shape)
df_orig = pd.read_csv(folder_raw_data_milling / "milling.csv.gz", compression='gzip')

(1157, 4507)


In [5]:
df_orig.head()

,cut_id,case,time,ae_spindle,ae_table,vib_spindle,vib_table,smcdc,smcac,tool_class
0,0_0,0,0.000,0.219727,0.272827,0.733643,2.116699,6.840820,0.124512,0
1,0_0,0,0.004,0.246582,0.322266,0.778809,2.277832,6.660156,-0.561523,0
2,0_0,0,0.008,0.294189,0.283813,0.758057,2.343750,6.508789,-2.099609,0
3,0_0,0,0.012,0.323486,0.260010,0.726318,2.448730,6.542969,-2.731934,0
4,0_0,0,0.016,0.290527,0.253296,0.653076,2.546387,6.621094,-3.505859,0


In [6]:
df.head()

,cut_id,vib_table__time_reversal_asymmetry_statistic__lag_1,vib_table__time_reversal_asymmetry_statistic__lag_2,vib_table__time_reversal_asymmetry_statistic__lag_3,vib_table__c3__lag_1,vib_table__c3__lag_2,vib_table__c3__lag_3,vib_table__cid_ce__normalize_True,vib_table__cid_ce__normalize_False,vib_table__symmetry_looking__r_0.0,...,vib_spindle__permutation_entropy__dimension_5__tau_1,vib_spindle__permutation_entropy__dimension_6__tau_1,vib_spindle__permutation_entropy__dimension_7__tau_1,vib_spindle__query_similarity_count__query_None__threshold_0.0,"vib_spindle__matrix_profile__feature_""min""__threshold_0.98","vib_spindle__matrix_profile__feature_""max""__threshold_0.98","vib_spindle__matrix_profile__feature_""mean""__threshold_0.98","vib_spindle__matrix_profile__feature_""median""__threshold_0.98","vib_spindle__matrix_profile__feature_""25""__threshold_0.98","vib_spindle__matrix_profile__feature_""75""__threshold_0.98"
0,0_0,0.059820,0.019726,0.090884,10.242404,9.998320,9.917406,8.316355,2.330515,0.0,...,3.528049,3.889565,3.964837,NaN,1.367334,2.665270,1.862989,1.863036,1.561820,2.097634
1,0_1,-0.032651,0.114725,-0.076326,10.013332,9.918871,10.131679,8.968916,2.595267,0.0,...,3.684120,3.951186,4.012640,NaN,0.909485,2.750737,1.812363,1.724483,1.530175,2.259445
2,0_10,0.062789,0.124616,0.091206,9.749092,9.611839,9.683704,8.938175,2.411882,0.0,...,3.582979,3.857200,3.955815,NaN,0.955919,2.509319,1.498419,1.466404,1.262464,1.646470
3,0_11,-0.041695,-0.085356,-0.074667,9.335468,9.284036,9.482934,8.809535,2.613744,0.0,...,3.661015,3.927690,4.036541,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0_12,0.076698,0.140736,0.045914,9.450062,9.291278,9.362848,8.643183,2.258746,0.0,...,3.666678,3.904193,4.036541,NaN,0.810572,1.879254,1.327153,1.330487,1.216766,1.457333
